In [3]:
%load_ext autoreload
%autoreload 2

In [30]:
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.wcs.utils import wcs_to_celestial_frame
from astropy.table import Table
from astropy.nddata import Cutout2D
from astropy.io.fits.hdu.image import PrimaryHDU
import astropy.units as u

import hda_fits.fits as hfits
from hda_fits.fits import column_dtype_byte_to_string, read_shimwell_catalog

plt.rcParams["figure.figsize"] = (12, 8)

In [ ]:
def write_pink_file_v2_header(filepath: str, number_of_images: int, image_height: int, image_width: int):
    with open(filepath, 'wb') as f:
        f.write(struct.pack('i' * 8, 2, 0, 0, number_of_images, 0, 2, image_height, image_width))

def write_pink_file_v2_data(filepath, data: np.array):
    with open(filepath, 'ab') as f:
        f.write(struct.pack('%sf' % data.size, *data.tolist()))

def write_mosaic_objects_to_pink_file_v2(filepath: str, hdu: PrimaryHDU, coordinates: List[WCSCoordinates], size: Union[int, RectangleSize]):
    write_pink_file_v2_header(filepath, 10, 200, 200)
    for coord in coords:
        data = hfits.create_cutout2D_as_flattened_numpy_array(hdu, coord, 200)
        write_pink_file_v2_data(filepath, data)


## Create table subset including P205+55 entries

In [2]:
tab = read_shimwell_catalog("../data/LOFAR_HBA_T1_DR1_catalog_v1.0.srl.fits", reduced=True)
tab = tab[tab.Mosaic_ID.str.contains("P205")].loc[:, ["Source_Name", "RA", "DEC", "Mosaic_ID"]]

In [4]:
tab

,Source_Name,RA,DEC,Mosaic_ID
182259,ILTJ134917.04+535737.2,207.320991,53.960334,P205+55
182260,ILTJ134916.45+535801.7,207.318522,53.967151,P205+55
182261,ILTJ134915.23+540433.5,207.313443,54.075982,P205+55
182262,ILTJ134915.03+540812.1,207.312622,54.136707,P205+55
182263,ILTJ134911.34+540239.4,207.297256,54.044286,P205+55
...,...,...,...,...
189028,ILTJ134000.89+562526.5,205.003718,56.424033,P205+55
189029,ILTJ133759.61+550106.4,204.498375,55.018459,P205+55
189030,ILTJ133614.95+544711.3,204.062292,54.786475,P205+55
189031,ILTJ133340.22+540139.8,203.417603,54.027725,P205+55


In [8]:
hdu = hfits.load_mosaic("P205+55", "../data/")[0]

2021-10-29 22:32:54,092 fits.py: Loading ../data/P205+55-mosaic.fits


In [24]:
tab = tab.reset_index(drop=True)
coords = tab.loc[:10, ["RA", "DEC"]].values.tolist()

In [31]:
arr = hfits.create_cutout2D_as_flattened_numpy_array(hdu, coords[0], 200)
arr

array([-1.98267850e-05,  6.28855382e-07,  8.91231630e-06, ...,
        6.10927654e-05,  6.73056855e-05,  1.35995937e-04])

In [36]:
arr.astype("float32").tofile("yep.bin")

In [41]:
arr.size

40000

In [42]:
from hda_fits import pink

In [ ]:
pink.write_mosaic_objects_to_pink_file_v2("here")